In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import nengo
import nengolib

ImportError: cannot import name 'with_metaclass'

In [4]:
dt = 0.001

class System(object):
    def __init__(self):
        self.value = 0.0
    def update(self, x):
        self.value -= dt * x * 20
        self.value = np.clip(self.value, -1, 1)
        return self.value
    def make_node(self):
        return nengo.Node(lambda t, x: self.update(x), size_in=1, size_out=1)
        
class Target(object):
    def __init__(self):
        self.target = 0.5
        self.chosen = 0
    def update_target(self, x):
        self.target = x
    def make_target_node(self):
        return nengo.Node(lambda t, x: self.update_target(x), size_in=1, size_out=0)
    def make_chosen_node(self):
        return nengo.Node(lambda t: self.chosen)
    
target = Target()
        
sys = System()
        
N = 3
times = np.linspace(0, 0.1, N)
#times = [0]
model = nengo.Network()
with model:
    system = sys.make_node()
    
    targ = target.make_target_node()
    #ctrl_target = nengo.Node(lambda t: np.sign(np.sin(t*2*np.pi*2))*0.5)
    ctrl_target = nengo.Node(lambda t: np.sin(t*2*np.pi*2)*0.5)
    
    nengo.Connection(ctrl_target, targ, synapse=None)
    choice = target.make_chosen_node()
    
    predictor = nengo.Ensemble(n_neurons=500, dimensions=2, neuron_type=nengo.LIFRate())
    nengo.Connection(system, predictor[0], synapse=0)
    
    #ctrl = nengo.Node(np.cos)
    
    nengo.Connection(choice, system, synapse=None)
    nengo.Connection(choice, predictor[1], synapse=None)
    
    future = nengo.Node(None, size_in=len(times))
    
    
    conns = []
    probes = []
    for i, t in enumerate(times):
        def ideal(x, t=t):
            return 0
            theta = np.arctan2(x[1], x[0])
            return np.cos(theta-t)
        
        c = nengo.Connection(predictor, future[i], function=ideal,
                             synapse=None,
                             learning_rule_type=nengo.PES(
                                                          pre_synapse=t,
                                                          #pre_synapse=nengolib.synapses.DiscreteDelay(int(t/dt)),
                                                          learning_rate=1e-4))
        conns.append(c)
        probes.append(nengo.Probe(c, 'weights'))        
    
    error = nengo.Node(None, size_in=len(times))
    nengo.Connection(system, error, transform=-np.ones((len(times), 1)), synapse=None)
    for i, t in enumerate(times):
        nengo.Connection(future[i], error[i], 
                         synapse=t
                         #synapse=nengolib.synapses.DiscreteDelay(int(t/dt))
                        )
        nengo.Connection(error[i], conns[i].learning_rule, synapse=None)
        
        
    result = nengo.Node(None, size_in=2)
    nengo.Connection(system, result[0], synapse=None)
    nengo.Connection(ctrl_target, result[1])
    
    p_result = nengo.Probe(result)
    

        #import nengo_learning_display
        #plot = nengo_learning_display.Plot1D(c, 
        #            domain=[(np.cos(t), np.sin(t)) for t in np.linspace(-np.pi, np.pi, 30)], 
        #            range=(-1.5,1.5))
        #plot.label = 'plot predict %g' % t
        
    
def on_step(sim):
    #plot.update(sim)
    
    domain = np.array([[sys.value, x] for x in np.linspace(-1, 1, 20)])
    _, a = nengo.utils.ensemble.tuning_curves(predictor, sim, domain)    
    
    w = np.vstack([sim._probe_outputs[p][-1] for p in probes])
    for p in probes:
        del sim._probe_outputs[p][:]
    y = np.dot(a, w.T)
    
    loss = np.mean((y - target.target)**2, axis=1)
    
    index = np.argmin(loss)
    target.chosen = domain[index,1]

#sim = nengo.Simulator(model)
#sim.run(0.1)


In [5]:
import nengo_gui.jupyter
nengo_gui.jupyter.InlineGUI(model, cfg='mdp.cfg')

c:\users\terry\documents\github\nengo_gui\nengo_gui\jupyter.py:70: ConfigReuseWarning: Reusing config. Only the most recent visualization will update the config.
  "Reusing config. Only the most recent visualization will "
